# Data Processing Scripts for PMFP

In [3]:
import io
import boto3
import pandas as pd

def read_feather_file_from_s3(s3_url):
    assert s3_url.startswith("s3://")
    bucket_name, key_name = s3_url[5:].split("/", 1)

    s3 = boto3.client('s3')
    retr = s3.get_object(Bucket=bucket_name, Key=key_name)
    
    return pd.read_feather(io.BytesIO(retr['Body'].read()))

In [12]:
# FILES = ["s3://pmpf-data/iot_pmfp_data.feather", ]
# NAMES = ["data.csv"]
FILES = ["s3://pmpf-data/iot_pmfp_labels.feather"]
NAMES = ["labels.csv"]

In [14]:
s3 = boto3.resource('s3')

for file, name in zip(FILES, NAMES):
    df = read_feather_file_from_s3(file)
    df.to_csv(name)
    s3.meta.client.upload_file(name, 'pmpf-data', name)